## NTB Reach Modeling with impressions as target on ntb view attributed sold items data

### data Load

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file to explore its content and structure
campaign_data = pd.read_csv('Amazon Ads All.csv')

# Display the first few rows of the dataframe and the column names to understand the dataset
campaign_data.head(), campaign_data.columns

(   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
 0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
 1   66528  3003509  6064714310  offsite_mobileapp            4       0   
 2   88613  9302679  7428981046    offsite_desktop            4       0   
 3   57975  3325755  7489730099    offsite_desktop           13       0   
 4   13386  6532070  6004552148  offsite_mobileweb          246       0   
 
    auction_cost  adjusted_cost hit_day_utc  \
 0         10.89          8.790    02/05/24   
 1          0.08          0.032    02/05/24   
 2          0.08          0.032    02/05/24   
 3          0.26          0.127    02/05/24   
 4          2.46          1.978    02/05/24   
 
                             targeting_secondary                 vertical  \
 0  category=Automotive Replacement Control Arms               Automotive   
 1              category=Canned & Jarred Seafood  Consumer Packaged Goods   
 2                       category

### Cleaning

In [2]:
## CLEANING
# Attempt to parse dates in the '%d/%m/%y' format
campaign_data['hit_day_utc_parsed'] = pd.to_datetime(campaign_data['hit_day_utc'], format='%d/%m/%y', errors='coerce')

# Parse remaining unparsed dates in the '%Y-%m-%d %H:%M:%S' format
mask = campaign_data['hit_day_utc_parsed'].isna()
campaign_data.loc[mask, 'hit_day_utc_parsed'] = pd.to_datetime(campaign_data.loc[mask, 'hit_day_utc'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check for any remaining non-parsed dates
remaining_non_parsed = campaign_data[campaign_data['hit_day_utc_parsed'].isna()]
if not remaining_non_parsed.empty:
    print("Remaining non-parsed dates:")
    print(remaining_non_parsed)


# Extract month and day of the week from parsed dates
campaign_data['month_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%B')  # Get month names
campaign_data['day_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%A')    # Get day names

# Fill missing values in 'vertical' and 'sub_vertical' with 'Unknown'
campaign_data['vertical'].fillna('Unknown', inplace=True)
campaign_data['sub_vertical'].fillna('Unknown', inplace=True)

# Identify the last 6 columns
last_6_columns = campaign_data.columns[-6:]

# Fill missing values in the last 6 columns with 0
campaign_data[last_6_columns] = campaign_data[last_6_columns].fillna(0)

# Display the cleaned dataset
print("Cleaned Campaign Data Sample:")
print(campaign_data.head())

# filtering the data to focus on only new to brand view attributed units sold
campaign_data = campaign_data[(campaign_data['ntb_view_attributed_units_sold'] > 0)]

/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_65883/2625150685.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  campaign_data['vertical'].fillna('Unknown', inplace=True)
/var/folders/80/ghdt2w012_x2yg018vx45m300000gn/T/ipykernel_65883/2625150685.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting 

Cleaned Campaign Data Sample:
   ADV_ID    AD_ID         SKU     placement_slot  impressions  clicks  \
0   84373  6370624  9985955750  offsite_mobileweb         1089       0   
1   66528  3003509  6064714310  offsite_mobileapp            4       0   
2   88613  9302679  7428981046    offsite_desktop            4       0   
3   57975  3325755  7489730099    offsite_desktop           13       0   
4   13386  6532070  6004552148  offsite_mobileweb          246       0   

   auction_cost  adjusted_cost hit_day_utc  \
0         10.89          8.790    02/05/24   
1          0.08          0.032    02/05/24   
2          0.08          0.032    02/05/24   
3          0.26          0.127    02/05/24   
4          2.46          1.978    02/05/24   

                            targeting_secondary  ...  \
0  category=Automotive Replacement Control Arms  ...   
1              category=Canned & Jarred Seafood  ...   
2                       category=Comedy & Humor  ...   
3                  categ

### Considering only categories and slots


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns


# Define features and target for NTB reach
features_ntb = [ 'targeting_secondary', 'placement_slot']
target_ntb = 'impressions'  # You can change this to 'ntb_view_attributed_units_sold' if desired

# Prepare the data
X_ntb = ntb_data[features_ntb]
y_ntb = ntb_data[target_ntb]

# One-hot encode categorical features
preprocessor_ntb = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', categories='auto'), ['targeting_secondary', 'placement_slot'])
    ])

# Define and train the Gradient Boosting Regressor for NTB reach
model_ntb = Pipeline(steps=[
    ('preprocessor', preprocessor_ntb),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

X_train_ntb, X_test_ntb, y_train_ntb, y_test_ntb = train_test_split(X_ntb, y_ntb, test_size=0.2, random_state=42)
model_ntb.fit(X_train_ntb, y_train_ntb)

# Predict and evaluate the NTB reach model
y_pred_ntb = model_ntb.predict(X_test_ntb)
mse_ntb = mean_squared_error(y_test_ntb, y_pred_ntb)
mae_ntb = mean_absolute_error(y_test_ntb, y_pred_ntb)
r2_ntb = r2_score(y_test_ntb, y_pred_ntb)

# Display the evaluation metrics for NTB reach model
print(f"NTB Reach Model - Mean Squared Error: {mse_ntb}")
print(f"NTB Reach Model - Mean Absolute Error: {mae_ntb}")
print(f"NTB Reach Model - R-squared: {r2_ntb}")

# Feature importance for NTB reach model
importance_ntb = model_ntb.named_steps['regressor'].feature_importances_
feature_names_ntb = model_ntb.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out([ 'targeting_secondary', 'placement_slot'])
feature_importance_ntb = pd.DataFrame({
    'Feature': feature_names_ntb,
    'Importance': importance_ntb
}).sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
print("Feature Importance for NTB Reach Model:")
print(feature_importance_ntb)



NameError: name 'ntb_data' is not defined

### Considering other features to maximize impressions

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np


# Attempt to parse dates in the '%d/%m/%y' format
campaign_data['hit_day_utc_parsed'] = pd.to_datetime(campaign_data['hit_day_utc'], format='%d/%m/%y', errors='coerce')

# Parse remaining unparsed dates in the '%Y-%m-%d %H:%M:%S' format
mask = campaign_data['hit_day_utc_parsed'].isna()
campaign_data.loc[mask, 'hit_day_utc_parsed'] = pd.to_datetime(campaign_data.loc[mask, 'hit_day_utc'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Check for any remaining non-parsed dates
remaining_non_parsed = campaign_data[campaign_data['hit_day_utc_parsed'].isna()]
if not remaining_non_parsed.empty:
    print("Remaining non-parsed dates:")
    print(remaining_non_parsed)


# Extract month and day of the week from parsed dates
campaign_data['month_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%B')  # Get month names
campaign_data['day_name'] = campaign_data['hit_day_utc_parsed'].dt.strftime('%A')    # Get day names

# Fill missing values
campaign_data.fillna({
    'vertical': 'unknown',
    'sub_vertical': 'unknown',
    'ntb_click_attributed_units_sold': 0,
    'ntb_view_attributed_units_sold': 0,
    'ntb_click_attributed_orders': 0,
    'ntb_view_attributed_orders': 0,
    'view_attributed_units_sold': 0,
    'view_attributed_orders': 0
}, inplace=True)

# Filter data for NTB reach
ntb_data = campaign_data[(campaign_data['ntb_view_attributed_units_sold'] > 0)]

# Define features and target for NTB reach
features_ntb = ['targeting_secondary', 
                'placement_slot', 'month_name', 'day_name']
target_ntb = 'impressions'

# Prepare the data
X_ntb = ntb_data[features_ntb]
y_ntb = ntb_data[target_ntb]

# One-hot encode categorical features
preprocessor_ntb = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['targeting_secondary', 'placement_slot', 'month_name', 'day_name'])
    ],
    remainder='passthrough'
)

# Define and train the Gradient Boosting Regressor for NTB reach
model_ntb = Pipeline(steps=[
    ('preprocessor', preprocessor_ntb),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

X_train_ntb, X_test_ntb, y_train_ntb, y_test_ntb = train_test_split(X_ntb, y_ntb, test_size=0.2, random_state=42)
model_ntb.fit(X_train_ntb, y_train_ntb)

# Predict and evaluate the NTB reach model
y_pred_ntb = model_ntb.predict(X_test_ntb)
mse_ntb = mean_squared_error(y_test_ntb, y_pred_ntb)
mae_ntb = mean_absolute_error(y_test_ntb, y_pred_ntb)
r2_ntb = r2_score(y_test_ntb, y_pred_ntb)

# Display the evaluation metrics for NTB reach model
print(f"NTB Reach Model - Mean Squared Error: {mse_ntb}")
print(f"NTB Reach Model - Mean Absolute Error: {mae_ntb}")
print(f"NTB Reach Model - R-squared: {r2_ntb}")

# Get feature names after one-hot encoding
all_feature_names = model_ntb.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(['targeting_secondary', 'placement_slot', 'month_name', 'day_name'])
# all_feature_names = np.concatenate((encoded_feature_names, [ 'adjusted_cost']))

# Calculate feature importance
importance_ntb = model_ntb.named_steps['regressor'].feature_importances_
feature_importance_ntb = pd.DataFrame({
    'Feature': all_feature_names,
    'Importance': importance_ntb
}).sort_values(by='Importance', ascending=False)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
print("Feature Importance for NTB Reach Model:")
print(feature_importance_ntb)


NTB Reach Model - Mean Squared Error: 8475900.0519299
NTB Reach Model - Mean Absolute Error: 1791.8868942504591
NTB Reach Model - R-squared: 0.613637876147813
Feature Importance for NTB Reach Model:
                                                                            Feature  \
909                                                placement_slot_offsite_mobileapp   
859                              targeting_secondary_category=Women's Running Shoes   
319                                  targeting_secondary_category=Foundation Makeup   
673                            targeting_secondary_category=Sheet & Pillowcase Sets   
394                                     targeting_secondary_category=Home & Kitchen   
567                                 targeting_secondary_category=Oral Care Products   
900                                 targeting_secondary_content-category=Soap Opera   
463                                  targeting_secondary_category=Kids' Electronics   
222            tar

In [5]:
pd.set_option('display.max_colwidth', 50)
print("Feature Importance for NTB Reach Model:")
print(feature_importance_ntb)

Feature Importance for NTB Reach Model:
                                               Feature  Importance
909                   placement_slot_offsite_mobileapp    0.383660
859  targeting_secondary_category=Women's Running S...    0.236025
319     targeting_secondary_category=Foundation Makeup    0.060768
673  targeting_secondary_category=Sheet & Pillowcas...    0.044100
394        targeting_secondary_category=Home & Kitchen    0.029773
567    targeting_secondary_category=Oral Care Products    0.028157
900    targeting_secondary_content-category=Soap Opera    0.026008
463     targeting_secondary_category=Kids' Electronics    0.020559
222  targeting_secondary_category=Desk Accessories ...    0.018556
912                                month_name_February    0.014751
691  targeting_secondary_category=Single-Serve Coff...    0.014673
913                                   month_name_March    0.014432
641  targeting_secondary_category=Probiotic Nutriti...    0.012958
908                   

Tuned model